In [1]:
# Fist install the library

#%pip install aepy

In [2]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np

from sklearn.metrics import accuracy_score
from keras_core import utils

from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate, display_diff, add_noise
from aepy.models.vq_vae.vq_vae_model import VQ_VAE
from aepy.models.base.default_architectures import Encoder_Conv_VQ_MNIST, Decoder_Conv_VQ_MNIST
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-12-07 13:46:19.548906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-07 13:46:23 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-07 13:46:23 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-07 13:46:23 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-07 13:46:23 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [4]:
train_data = dict(data=x_train.astype(float), labels=x_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
# Model creation
model = VQ_VAE(input_dim=(x_train.shape[1], x_train.shape[2]), 
            latent_dim=2, encoder=Encoder_Conv_VQ_MNIST, decoder=Decoder_Conv_VQ_MNIST, layers_conf=[32, 64])

model.jit_compile = False

128
2
(28, 28)


In [7]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=10)

trained_model = pipe(train_data=train_data)

2023-12-07 13:46:24 [INFO]: +++ training_pipeline +++
2023-12-07 13:46:24 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-07_13-46-24


Epoch 1/10


/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  return py_builtins.overload_of(f)(*args)


(128, 7, 7, 2)
Tensor("vector_quantizer_1/Shape:0", shape=(4,), dtype=int32)
(6272, 2)
Quantizarion process
(6272,)
Tensor("vector_quantizer_1/one_hot:0", shape=(6272, 128), dtype=float32)
Tensor("vector_quantizer_1/MatMul_1:0", shape=(6272, 2), dtype=float32)
Pre-reshape
Post-reshape
(128, 7, 7, 2)
Tensor("vector_quantizer_1/Mean:0", shape=(), dtype=float32)
Tensor("vector_quantizer_1/Mean_1:0", shape=(), dtype=float32)
Post-add_loss
Last step
(128, 7, 7, 2)
[]
(128, 7, 7, 2)
Tensor("vq_vae_1/vector_quantizer_1/Shape:0", shape=(4,), dtype=int32)
(6272, 2)
Quantizarion process
(6272,)
Tensor("vq_vae_1/vector_quantizer_1/one_hot:0", shape=(6272, 128), dtype=float32)
Tensor("vq_vae_1/vector_quantizer_1/MatMul_1:0", shape=(6272, 2), dtype=float32)
Pre-reshape
Post-reshape
(128, 7, 7, 2)
Tensor("vq_vae_1/vector_quantizer_1/Mean:0", shape=(), dtype=float32)
Tensor("vq_vae_1/vector_quantizer_1/Mean_1:0", shape=(), dtype=float32)
Post-add_loss
Last step
(128, 7, 7, 2)
[<tf.Tensor 'vq_vae_1/ve